steps:

1. import pdf doc
2. process text for embedding(split into chunks of senten)
3. embed text chunk with embedding model 
4. Save embedding to file for later

In [3]:
import os 
import requests

# get pdf doc 
pdf_path = "human-nutrition-text.pdf"

if not os.path.exists(pdf_path):
    print(f"[info] file doesn't exists, downloading...")

    url =" https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

    response = requests.get(url)

    if response.status_code == 200:
        with open(pdf_path,"wb") as file:
            file.write(response.content)
        print(f"[INFO] file has downloaded ")
    else:
        print(f"Failed to download file {response.status_code}")
else:
    print(f"file {pdf_path} exists")

file human-nutrition-text.pdf exists


In [4]:
import fitz  # Ensure this import is at the top
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    cleaned_text = text.replace('\n', " ").strip()
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_text = []
    for page_number, page in tqdm(enumerate(doc)):  
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_text.append({
            "page_number": page_number-41,  # Corrected page numbering
            "page_char_count": len(text),
            "page_word_count": len(text.split(" ")),
            "page_sentence_count_raw": len(text.split(". ")),
            "page_token_count": len(text)/4,  # Assuming tokens are words; corrected calculation
            "text": text
        })

    return pages_and_text



In [6]:
pages_and_texts  = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [7]:
import random 
random.sample(pages_and_texts,k=3)

[{'page_number': 457,
  'page_char_count': 1597,
  'page_word_count': 271,
  'page_sentence_count_raw': 15,
  'page_token_count': 399.25,
  'text': 'details on food composition data, go to the USDA Food Composition  Databases page.  An Organism Requires Energy and Nutrient  Input  Energy is required in order to build molecules into larger  macromolecules (like proteins), and to turn macromolecules into  organelles and cells, which then turn into tissues, organs, and organ  systems, and finally into an organism. Proper nutrition provides  the necessary nutrients to make the energy that supports life’s  processes. Your body builds new macromolecules from the  nutrients in food.  Nutrient and Energy Flow  Energy is stored in a nutrient’s chemical bonds. Energy comes from  sunlight, which plants capture and, via photosynthesis, use it to  transform carbon dioxide in the air into the molecule glucose. When  the glucose bonds are broken, energy is released. Bacteria, plants,  and animals (in

In [8]:
import pandas as pd 
df=pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...


Token Count 
1. Emebdding model dont deal in infinite token 
2. LLM dont deal wiht infite token 

Some embedding model may been trained to embed seqence of 384 token into numerical space. 